In [7]:
from typing import Generator, Iterable, Dict, List
import datetime
import tqdm
from decimal import Decimal
import numpy as np
from dateutil import parser as date_parser
from sklearn.model_selection import ParameterGrid, ParameterSampler
import json

from google.cloud import bigquery

from more_itertools import chunked


In [8]:
bq = bigquery.Client()

In [9]:
def sample_execution_datetimes() -> List[datetime.datetime]:
    query = '''
    SELECT MIN(`timestamp`) AS `start_date`, MAX(`timestamp`) AS `end_date`
    FROM `trading_terminal_poc.coinbase_raw_l2_order_book`
    '''
    result = bq.query(query).result()
    df = result.to_dataframe()

    start_date, end_date = df.iloc[0].tolist()

    start_datetime = start_date.to_pydatetime()
    end_datetime = end_date.to_pydatetime()

    n_day = (end_datetime - start_datetime).days
    n_sample_per_day = 24

    rand = np.random.RandomState(17)
    random_timestamps = rand.uniform(start_datetime.timestamp(),
                                            end_datetime.timestamp(),
                                            size=n_day * n_sample_per_day,
                                            )

    sample_datetimes = list(map(datetime.datetime.fromtimestamp, random_timestamps))
    return sample_datetimes

In [10]:
symbols = ["BTC-USD", "SOL-USD", "ETH-USD",
            "AVAX-USD", "ADA-USD", "LTC-USD", 
            "ATOM-USD", "MATIC-USD", 
            "LINK-USD", "XLM-USD", 
            "DOT-USD",  "BCH-USD", 
            ]
sampled_datetimes = sample_execution_datetimes()

In [11]:
n_sample = 4320 * 10
param_sampler = ParameterSampler({
    'symbol': symbols,
    'side': ['buy', 'sell'],
    'start_execution_datetime': sampled_datetimes,
    'max_processing_time': [60*60],  # 1 Hour
    'percentage': [0.001, 0.0025, 0.0075, 0.025, 0.075, 0.1],
    'acceptable_slippage': [0.0025, 0.0075, 0.025, 0.05],
    'delay': [1, 2, 5, 8, 10],
    'target_order_amount': [100_000, 300_000, 5_000_000]
}, n_iter=n_sample, random_state=7)



In [12]:
def build_job_inventory_payload(params) -> list:

    payloads = []
    load_dt = datetime.datetime.now().isoformat()
    for i, param in enumerate(params):
        payload = {
            'id': i,
            'algorithm': 'PoV',
            'param': json.dumps(param, default=str),
            'status': 'CREATED',
            'load_dt': load_dt
        }
        payloads.append(payload)
    
    return payloads

In [13]:
payload_json = build_job_inventory_payload(param_sampler)

In [14]:
chunk_size = 256

for payload in tqdm.tqdm(chunked(payload_json, chunk_size), total=len(payload_json) / chunk_size):
    bq.insert_rows_json('trading_terminal_poc.job_inventory', payload)



100%|█████████▉| 168/168.75 [00:14<00:00, 12.65it/s]/Users/anythingth2/development/.python_virtual_env/cft_env/lib/python3.9/site-packages/tqdm/std.py:533: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 169/168.75 [00:14<00:00, 11.64it/s]
